Deadline: 24 novembre ore 23:59.
Solve efficiently a generic n^2-1 puzzle using path search algorithm.

Cost=  total number of actions you need to __evaluate__. An action is something that bring me to a new state. For example the number of swaps to do.

The result is the sequence of action that took you at the end. The goal is not to find a state but a sequence of actions from srtarting point to end point: we do not look for a soluzion but for a sequence of actions.

In [19]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappop, heappush

In [20]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

In [21]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

The state is a numpy array.

We created a function that returns the number of actions from a state pos1 to a state pos2.

Compute 100_000 random actions:

In [22]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 60669.25it/s]


array([[ 6,  8, 13,  2],
       [10,  9,  3,  7],
       [11,  4,  0,  1],
       [14, 12, 15,  5]])

Define a function that indicates if we end the search.

In [23]:
def test_goal(solution):
    arr_solution = np.reshape(solution, PUZZLE_DIM*PUZZLE_DIM)
    arr_solution_no_zero = arr_solution[0: len(arr_solution)-1]
    if np.all(arr_solution_no_zero[:-1] <= arr_solution_no_zero[1:]) and arr_solution[len(arr_solution)-1]==0:
        return True
    return False

# Depth search

Let's use a function that computes a value for each matrix so that it will be easy to store matrices.

This function shouuld be bi-directional: a number is associated to only one matrix and viceversa.

In [24]:
def matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += (10**i)*matrice[i]
    return int(score)


def simplified_matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += i*matrice[i]
    return int(score)

In [25]:
from collections import namedtuple, deque
import numpy as np

# Define action as a named tuple
action = namedtuple('Action', ['pos1', 'pos2'])

def depth_limited_search(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int) -> list[action] or None:
    """
    Performs a depth-limited search to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    - max_depth (int): The maximum depth limit for the search.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # # Backtrack if depth limit reached
        # if depth >= max_depth:
        #     continue
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            
            # Check if the next state has already been visited in the current path
            if matrix_score(next_state) not in visited:
                # Add the new state and path to stack, increase depth
                stack.append((next_state, path + [act], depth + 1))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(matrix_score(current_state))
    
    return None  # Return None if no solution is found within depth limit

# Iterative Deepening Depth-First Search (IDDFS) wrapper function
def iterative_deepening_dfs(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int = 50) -> list[action] or None:
    """
    Performs an iterative deepening DFS to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    - max_depth (int): The maximum depth to limit the search for IDDFS.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    for depth in range(1, max_depth + 1):
        result = depth_limited_search(initial_state, final_state, depth)
        if result is not None:
            return result
    return None


# Updated Depth search

At each iteration, among all the possible we choose the one with the minimum manahattan distance to the test goal.
In case there is no state with less manhattan distance compared to the current we select the oen that is the "best" among the possibilities. In case they are the same we use the last one.

In [26]:
def update_depth(initial_state: np.ndarray, final_state: np.ndarray) -> list or None:
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and calculate Manhattan distances for all possible moves
        successors = []
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_score = matrix_score(next_state)
            
            # Check if the next state has already been visited in the current path
            if next_score not in visited:
                # Calculate the Manhattan distance from the goal
                manhattan_dist = manhattan_distance(next_state, final_state)
                # Append (next_state, path + [act], depth + 1, manhattan_dist) to successors list
                successors.append((next_state, path + [act], depth + 1, manhattan_dist))
        
        # Sort successors by Manhattan distance (ascending)
        successors.sort(key=lambda x: x[3])  # Sort by the fourth item, which is manhattan_dist
        
        # Add sorted successors to the stack
        for next_state, new_path, new_depth, _ in successors:
            stack.append((next_state, new_path, new_depth))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(current_score)  # Uncomment if backtracking is implemented
    
    return None  # Return None if no solution is found within depth limit


# A*
In the context of the n-puzzle problem, **Manhattan distance** is a heuristic used to estimate how close a puzzle state is to the goal. For two matrices (representing the current state and the goal state), Manhattan distance is calculated by summing up the individual distances of each tile from its target position.

1. **Tile Distance**: For each tile, the distance is the absolute difference between its current row and target row, plus the absolute difference between its current column and target column.
   
2. **Total Distance**: Summing these values for all tiles gives the Manhattan distance for the puzzle. This value represents the minimum number of moves required to reach the goal state if we could move each tile independently.

In [27]:
def manhattan_distance(state: np.ndarray, goal_state: np.ndarray) -> int:
    """
    Calculates the Manhattan distance for each tile in the puzzle.
    """
    distance = 0
    for value in range(1, state.size):  # Skip 0 (empty space)
        target_pos = np.argwhere(goal_state == value)[0]
        current_pos = np.argwhere(state == value)[0]
        distance += abs(target_pos[0] - current_pos[0]) + abs(target_pos[1] - current_pos[1])
    return distance

In [28]:
def a_star(initial_state: np.ndarray, final_state: np.ndarray) -> list[action] or None:
    """
    Performs an A* search to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (manhattan_distance(initial_state, final_state), 0, initial_state.tobytes(), []))
    #this function push into open_set a new element continuing to garantee the heap properties.
    # the element that we insert are 4.
    
    visited = set()  
    
    while open_set:
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        
        # Check if we reached the goal
        if np.array_equal(current_state, final_state):
            return path
        
        # Mark current state as visited
        visited.add(current_bytes)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_bytes = next_state.tobytes()
            if next_bytes in visited:
                continue
            
            # Calculate scores for the next state
            new_g_score = g_score + 1
            new_f_score = new_g_score + manhattan_distance(next_state, final_state)
            
            # Push the new state into the priority queue with updated scores
            heappush(open_set, (new_f_score, new_g_score, next_bytes, path + [act]))
    
    return None  # Return None if no solution is found

In this code we are storing all the state and comparing them. This may be difficoult. Let's try to use the function that computes a singol number for a given state.

In [29]:
def a_star_no_storing(initial_state: np.ndarray, final_state: np.ndarray) -> list[action] or None:
    """
    Performs an A* search to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (manhattan_distance(initial_state, final_state), 0, initial_state.tobytes(), []))
    #this function push into open_set a new element continuing to garantee the heap properties.
    # the element that we insert are 4.
    
    visited = set()  
    optimum = matrix_score(final_state)
    
    while open_set:
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # Mark current state as visited
        visited.add(current_score)
        #visited.add(current_bytes)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_bytes = next_state.tobytes()
            value = matrix_score(next_state)
            if value in visited:
                continue
            
            # Calculate scores for the next state
            new_g_score = g_score + 1
            new_f_score = new_g_score + manhattan_distance(next_state, final_state)
            
            # Push the new state into the priority queue with updated scores
            heappush(open_set, (new_f_score, new_g_score, next_bytes, path + [act]))
    
    return None  # Return None if no solution is found

The problem is that still we are not able to solve 4*4 puzzle.

In [30]:
def state_to_tuple(state: np.ndarray) -> tuple:
    return tuple(state.flatten())

def linear_conflict(state, goal):
    conflict = 0
    size = state.shape[0]
    for row in range(size):
        row_goal = goal[row]
        row_state = state[row]
        for i in range(size):
            for j in range(i + 1, size):
                if row_state[i] in row_goal and row_state[j] in row_goal and row_state[i] > row_state[j]:
                    conflict += 2
    return conflict

def enhanced_heuristic(state, goal):
    return manhattan_distance(state, goal) + linear_conflict(state, goal)


def enhanced_a_star(initial_state: np.ndarray, final_state: np.ndarray) -> list[action] or None:
    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (enhanced_heuristic(initial_state, final_state), 0, initial_state.tobytes(), []))
    visited = set()  
    optimum = state_to_tuple(final_state)
    
    while open_set:
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        current_score = state_to_tuple(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # Mark current state as visited
        visited.add(current_score)
        #visited.add(current_bytes)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_bytes = next_state.tobytes()
            value = state_to_tuple(next_state)
            if value in visited:
                continue
            
            # Calculate scores for the next state
            new_g_score = g_score + 1
            new_f_score = new_g_score + enhanced_heuristic(next_state, final_state)
            
            # Push the new state into the priority queue with updated scores
            heappush(open_set, (new_f_score, new_g_score, next_bytes, path + [act]))
    
    return None  # Return None if no solution is found

In [31]:
class PuzzleHeuristicService:
    def __init__(self, goal_state: np.ndarray):
        self.goal_state = goal_state

    def manhattan_distance(self, state: np.ndarray) -> int:
        size = state.shape[0]
        dist = 0
        for i in range(size):
            for j in range(size):
                value = state[i, j]
                if value != 0:  # Ignora il tassello vuoto
                    target_x, target_y = divmod(np.where(self.goal_state.flatten() == value)[0][0], size)
                    dist += abs(i - target_x) + abs(j - target_y)
        return dist

    def linear_conflict(self, state: np.ndarray) -> int:
        conflict = 0
        size = state.shape[0]
        for row in range(size):
            row_goal = self.goal_state[row]
            row_state = state[row]
            for i in range(size):
                for j in range(i + 1, size):
                    if (
                        row_state[i] in row_goal
                        and row_state[j] in row_goal
                        and row_state[i] > row_state[j]
                    ):
                        conflict += 2
        return conflict

    def combined_heuristic(self, state: np.ndarray) -> int:
        return self.manhattan_distance(state) + self.linear_conflict(state)




def enhanced_a_star(initial_state: np.ndarray, final_state: np.ndarray) -> list or None:
    """
    Enhanced A* algorithm for the n-puzzle problem using modular heuristics.
    """
    # Classe per l'euristica
    heuristic_service = PuzzleHeuristicService(final_state)

    def calculate_heuristic(state: np.ndarray) -> int:
        # Usa l'euristica combinata (Manhattan + Linear Conflict)
        return heuristic_service.combined_heuristic(state)

    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (calculate_heuristic(initial_state), 0, initial_state.tobytes(), []))
    visited = set()
    optimum = state_to_tuple(final_state)

    while open_set:
        # Estrai il nodo con il punteggio più basso
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        current_score = state_to_tuple(current_state)

        # Controlla se il nodo corrente è il goal
        if current_score == optimum:
            return path  # Soluzione trovata

        # Aggiungi lo stato corrente ai visitati
        visited.add(current_score)

        # Genera e valuta le mosse possibili
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_score = state_to_tuple(next_state)
            if next_score in visited:
                continue

            # Calcola i punteggi per lo stato successivo
            new_g_score = g_score + 1
            new_f_score = new_g_score + calculate_heuristic(next_state)

            # Aggiungi il nuovo stato alla coda di priorità
            heappush(open_set, (new_f_score, new_g_score, next_state.tobytes(), path + [act]))

    return None  # Nessuna soluzione trovata


In [32]:
import numpy as np
from heapq import heappop, heappush

# Utility Functions
def state_to_tuple(state: np.ndarray) -> tuple:
    """Converte uno stato in una tupla immutabile."""
    return tuple(state.flatten())

def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    """Calcola la Manhattan Distance tra lo stato attuale e quello finale."""
    size = state.shape[0]
    distance = 0
    for x in range(size):
        for y in range(size):
            value = state[x, y]
            if value != 0:  # Ignora lo spazio vuoto
                goal_x, goal_y = divmod(np.where(goal == value)[0][0], size)
                distance += abs(goal_x - x) + abs(goal_y - y)
    return distance

def linear_conflict(state: np.ndarray, goal: np.ndarray) -> int:
    """Calcola le Linear Conflicts."""
    conflict = 0
    size = state.shape[0]
    for row in range(size):
        row_goal = goal[row]
        row_state = state[row]
        for i in range(size):
            for j in range(i + 1, size):
                if row_state[i] in row_goal and row_state[j] in row_goal and row_state[i] > row_state[j]:
                    conflict += 2
    return conflict

def enhanced_heuristic(state: np.ndarray, goal: np.ndarray) -> int:
    """Combina Manhattan Distance e Linear Conflict."""
    return manhattan_distance(state, goal) + linear_conflict(state, goal)

# Puzzle Solver
class EnhancedAStar:
    def __init__(self, initial_state: np.ndarray, final_state: np.ndarray):
        self.initial_state = initial_state
        self.final_state = final_state
        self.visited = set()
        self.goal_tuple = state_to_tuple(final_state)

    def solve(self):
        """
        Risolve il puzzle utilizzando una versione ottimizzata dell'algoritmo A*.
        :return: La sequenza di mosse per arrivare alla soluzione o None se non esiste.
        """
        open_set = []
        initial_heuristic = enhanced_heuristic(self.initial_state, self.final_state)
        heappush(open_set, (initial_heuristic, 0, self.initial_state.tobytes(), []))

        while open_set:
            f_score, g_score, current_bytes, path = heappop(open_set)
            current_state = np.frombuffer(current_bytes, dtype=self.initial_state.dtype).reshape(self.initial_state.shape)
            current_score = state_to_tuple(current_state)

            # Controlla se lo stato attuale è quello finale
            if current_score == self.goal_tuple:
                return path

            # Se lo stato è già stato visitato, salta
            if current_score in self.visited:
                continue
            self.visited.add(current_score)

            # Genera le mosse disponibili
            for action, next_state in self._get_available_actions(current_state):
                next_score = state_to_tuple(next_state)
                if next_score in self.visited:
                    continue

                # Calcola i punteggi per il nuovo stato
                g_new = g_score + 1
                f_new = g_new + enhanced_heuristic(next_state, self.final_state)

                # Aggiungi il nuovo stato alla coda di priorità
                heappush(open_set, (f_new, g_new, next_state.tobytes(), path + [action]))

        return None  # Nessuna soluzione trovata

    def _get_available_actions(self, state: np.ndarray):
        """Genera le mosse disponibili dallo stato corrente."""
        moves = []
        zero_pos = np.argwhere(state == 0)[0]
        directions = {
            'up': (-1, 0),
            'down': (1, 0),
            'left': (0, -1),
            'right': (0, 1)
        }

        for action, (dx, dy) in directions.items():
            new_pos = zero_pos + np.array([dx, dy])
            if 0 <= new_pos[0] < state.shape[0] and 0 <= new_pos[1] < state.shape[1]:
                next_state = state.copy()
                next_state[zero_pos[0], zero_pos[1]], next_state[new_pos[0], new_pos[1]] = \
                    next_state[new_pos[0], new_pos[1]], next_state[zero_pos[0], zero_pos[1]]
                moves.append((action, next_state))

        return moves


# Main

In [33]:
test_goal = np.arange(1, PUZZLE_DIM*PUZZLE_DIM, 1)
test_goal = np.append(test_goal, 0)
test_goal = test_goal.reshape((PUZZLE_DIM, PUZZLE_DIM))

# Try the upgrade depth search

In [34]:
#update_depth(state, test_goal)

# Try the A* algorithm

In [35]:
#a_star(state, test_goal)

In [36]:
# Stato iniziale e finale di esempio
initial = np.array([[5, 1, 2, 3],
                    [9, 6, 7, 4],
                    [13, 14, 11, 8],
                    [0, 10, 15, 12]])
final = np.array([[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12],
                  [13, 14, 15, 0]])

solver = EnhancedAStar(initial, final)

solution = solver.solve()

print("Solution:", solution)

KeyboardInterrupt: 

In [ ]:
#enhanced_a_star(state, test_goal)

KeyboardInterrupt: 